In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [43]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd

import torch.nn as nn
class Layer(nn.Module):#оголошення класу
    def __init__(self, inputs, outputs, activation_func=None):#виклик конструктора,активація по дефолту не задана
        super(Layer, self).__init__()#Виклик конструктора батьківського класу
        self.linear = nn.Linear(inputs, outputs)# Створення об'єкту лінійного шару з визначеними кількостями вхідних та вихідних вузлів
        self.activation_func = activation_func if activation_func is not None else nn.Identity()#Ініціалізація функції активації. Якщо вона задана, використовується передана функція активації, інакше використовується nn.Identity(), яка не змінює вхідний сигнал.

    def forward(self, x):#метод forward, який викликається під час прямого проходу через мережу.
        x = self.linear(x)#Застосування лінійного перетворення, представленого nn.Linear, до вхідного тензору x
        x = self.activation_func(x)#Застосування функції активації до результату лінійного перетворення
        return x#

class MLP(nn.Module):#оголошення класу
    def __init__(self, layer_sizes, activation_funcs=None):#конструктор приймає список розмірів шарів у мережі (layer_sizes) , список функцій активації для кожного шару (якщо є)
        super(MLP, self).__init__()#конструктора батьківського класу
        if activation_funcs is None:#Перевірка, чи заданий список функцій активації. Якщо він не заданий (None), встановлюється список з однаковими значеннями None, крім останнього шару (за замовчуванням функція активації не застосовується до останнього шару).
            activation_funcs = [None] * (len(layer_sizes) - 1)  # Default to no activation
        self.layers = nn.ModuleList([Layer(layer_sizes[i], layer_sizes[i+1], activation_funcs[i]) #створення шарів
                                     for i in range(len(layer_sizes) - 1)])#Цикл, який проходить по всім шарам крім останнього.

    def forward(self, x):#метод forward, який викликається під час прямого проходу через мережу.
        for layer in self.layers:#Подача вхідного тензору x через кожний шар мережі. Кожний шар обробляє вхід і повертає вихід, який стає новим вхідним тензором для наступного шару.
            x = layer(x)
        return x

# Завантаження датасету Iris
iris = load_iris()
X, y = iris.data, iris.target

# Скоригування масштабу даних
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Конвертація в тензори PyTorch
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# Розділення даних на тренувальний та тестовий набори
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# Створення об'єктів TensorDataset для тренувального та тестового наборів
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Створення Dataloader для тренувального та тестового наборів
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

def train(model, criterion, optimizer, train_loader, num_epochs=10):
    model.train()  # Встановлюємо модель у режим навчання
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()  # Обнуляємо градієнти
            outputs = model(inputs)  # Прямий прохід
            loss = criterion(outputs, labels)  # Обрахунок втрат
            loss.backward()  # Обчислення градієнтів
            optimizer.step()  # Оновлення ваг моделі

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

# Визначення моделі, критерію, оптимізатора та кількості епох(датасет розміром 64*4)

model = MLP([4, 16, 3], [torch.relu, torch.relu])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 20

# Навчання моделі
train(model, criterion, optimizer, train_loader, num_epochs)


Epoch 1, Loss: 1.1076
Epoch 2, Loss: 1.0978
Epoch 3, Loss: 1.0882
Epoch 4, Loss: 1.0784
Epoch 5, Loss: 1.0691
Epoch 6, Loss: 1.0598
Epoch 7, Loss: 1.0506
Epoch 8, Loss: 1.0416
Epoch 9, Loss: 1.0324
Epoch 10, Loss: 1.0237
Epoch 11, Loss: 1.0149
Epoch 12, Loss: 1.0067
Epoch 13, Loss: 0.9984
Epoch 14, Loss: 0.9905
Epoch 15, Loss: 0.9828
Epoch 16, Loss: 0.9756
Epoch 17, Loss: 0.9684
Epoch 18, Loss: 0.9615
Epoch 19, Loss: 0.9546
Epoch 20, Loss: 0.9480
